# Multi-asset forecasting with an LSTM

This notebook contains code for training and forecasting 22 stocks from the OMXS30 index
with an LSTM model.

Import data and create the dataloader

In [10]:
import pandas as pd
from data.MultiAssetDataset import MultiAssetDataset
from torch.utils.data import DataLoader

df = pd.read_csv("data/OMXS22_model_features_raw.csv", index_col = "Date", parse_dates = True)

tickers = df["Ticker"].unique().tolist()
features = ["Return","Volume","SMA20","EMA20","RSI14","ReturnVola20"]
window   = 60
horizon  = 1

train_stop = pd.Timestamp("2020-01-01")
val_stop   = pd.Timestamp("2023-12-31")

df_train = df[df.index < train_stop]
df_val   = df[(df.index >= train_stop) & (df.index < val_stop)]
df_test  = df[df.index >= val_stop]

ds_train = MultiAssetDataset(df_train, tickers, features, window, horizon)
ds_val   = MultiAssetDataset(df_val,   tickers, features, window, horizon)
ds_test  = MultiAssetDataset(df_test,  tickers, features, window, horizon)

train_loader = DataLoader(ds_train, batch_size=32, shuffle=True)
val_loader   = DataLoader(ds_val,   batch_size=32)
test_loader  = DataLoader(ds_test,  batch_size=32)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from multi_asset_models.MultiAssetLSTM import MultiAssetLSTM
from Train import Trainer

model = MultiAssetLSTM(
    n_assets=len(tickers),
    n_features=len(features),
    hidden_size=64,
    num_layers=2,
    dropout=0.1,
    horizon=horizon
)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 1) Träningsparametrar
num_epochs   = 50
patience     = 5        # stopp efter 5 epoker utan förbättring
best_val     = float("inf")
patience_ctr = 0
save_path    = "best_multiasset_lstm.pt"


Train epoch loss: 0.0010
Validation loss: 0.0005


In [ ]:
print(val_loss**0.5)

2.293912402617011
